In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

try:
    from PIL import Image
except ImportError:
    import Image

In [2]:
IMAGES_DIR_PATH="./random-cars-dataset/images"
LABELS_DIR_PATH="./random-cars-dataset/license-numbers"

In [10]:
import os
images=[f for f in os.listdir(IMAGES_DIR_PATH)]

In [4]:
img_path=f'{IMAGES_DIR_PATH}/'

#predicting labels for all test images
%cd yolov5
!python detect.py --weights ./runs/train/exp/weights/best.pt --source ../{img_path} --save-txt --exist-ok --nosave
%cd ..

C:\Users\HP\Desktop\ML\ANPR\yolov5
Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=True, img_size=640, iou_thres=0.45, name='exp', nosave=True, project='runs/detect', save_conf=False, save_txt=True, source='.././random-cars-dataset/images/', update=False, view_img=False, weights=['./runs/train/exp/weights/best.pt'])
Fusing layers... 
image 1/87 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars0.png: 352x640 1 license_plate, Done. (0.219s)
image 2/87 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars1.png: 416x640 1 license_plate, Done. (0.203s)
image 3/87 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars102.png: 512x640 1 license_plate, Done. (0.281s)
image 4/87 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars107.png: 384x640 1 license_plate, Done. (0.203s)
image 5/87 C:\Users\HP\Desktop\ML\ANPR\yolov5\..\random-cars-dataset\images\Cars108.png: 512x640 1 licen

YOLOv5  0da0ab0 torch 1.8.1+cpu CPU

Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPS


In [11]:
def get_yolo_label(f_name):
    #opening label file
    label_file=open(f'./yolov5/runs/detect/exp/labels/{f_name}.txt')

    #reading line
    s=label_file.readline().replace('\n','').split(' ')

    #******extracting bounding box********#
    s=[ float(i) for i in s]    #[conf center_x center_y width height] 

    bottom_x=((2*s[1]+s[3])/2)
    top_x=bottom_x-s[3]

    bottom_y=((2*s[2]+s[4])/2)
    top_y=bottom_y-s[4]

    top_x=int(top_x*width)
    bottom_x=int(bottom_x*width)

    top_y=int(top_y*height)
    bottom_y=int(bottom_y*height)
    #*************************************#

    return [top_x,top_y,bottom_x,bottom_y]
   

In [12]:
def pre_process(plate):
    # resize image to three times as large as original for better readability
    plate=cv2.resize(plate,None,fx=3,fy=3,interpolation=cv2.INTER_CUBIC)

    #converting to gray scale
    plate=cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)

    #gaussian blur
    plate=cv2.GaussianBlur(plate, (5,5), 0)

    #binarizing
    plate=cv2.threshold(plate, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)[1]

    # apply dilation to make regions more clear
    ret, thresh = cv2.threshold(plate, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
    rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    plate= cv2.dilate(thresh, rect_kern, iterations = 1)
    
    return plate

In [13]:
def segment_chars(plate):
    contours,_=cv2.findContours(plate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])
    chars=[]

    for cont in contours:
        x,y,w,h=cv2.boundingRect(cont)

        #w: width of character
        #h height of character

        #shape of plate
        height_of_plate,width_of_plate=plate.shape

        #.......removing noise........#

        #if char is too wide then skip
        if width_of_plate<2*w: continue        

        #if char is too short in height then skip
        if height_of_plate>5*h: continue

        #if char h/w ratio is less than 1
        if h/float(w)<0.5: continue

        #if area is less than 100 pixels
        if h*w<70: continue

        #.............................#

        #checking if contour inside already added contour
        f=0
        for c in chars:
            x1,y1,w1,h1=cv2.boundingRect(c)
            if x>x1 and x<x1+w1 and y>y1 and y<y1+h1:
                #contour not useful, mark flag to 1
                f=1
                break

        if f==1: continue


        #appending valid contour
        chars.append(cont)
    
    return chars

In [14]:
#utility function to convert number(0-35) to label(0-9, A-Z)
def to_label(n):
    if n<=9:
        return str(n)
    
    return chr(n-10+ord('A'))


from keras.models import load_model

def predict_label(chars,char_recognizer):
    label=""

    #iterating over contours
    for char in chars:
        x,y,w,h=cv2.boundingRect(char) #w: width of character, h: height of character

        roi=plate[y:y+h,x:x+w]

        #adding padding
        #roi=cv2.copyMakeBorder(roi,5,5,5,5,cv2.BORDER_CONSTANT,value=[0,0,0])

        #resizing image to model input format (28x28)
        roi=cv2.resize(roi,(28,28))

        #inverting image: black to white and vice-versa
        #roi = cv2.bitwise_not(roi)

        #converting roi into required format
        roi=roi.reshape(28,28,1)

        #predicting label using model
        pred=char_recognizer.predict(np.array([roi]))

        #finding label mith max probability
        pred=np.argmax(pred)

        label+=to_label(pred)
        
    return label

In [15]:
from difflib import SequenceMatcher
import jellyfish

def cal_score(label_pred,label_true):
    sim=SequenceMatcher(None,label_pred,label_true).ratio()
    edit_dist=jellyfish.levenshtein_distance(label_pred,label_true)
    
    return [sim,edit_dist]

def final_score(scores):
    avg_sim=0
    avg_ed=0
    for s in scores:
        avg_sim+=s[0]
        avg_ed+=s[1]
        
    avg_sim/=len(scores)
    avg_ed/=len(scores)
        
    print(f"Average similarity: {avg_sim}\n Average Edit Distance: {avg_ed}")
        

In [18]:
char_recognizer=load_model('char_recog_lenet_model.h5')

scores=[]
for i,image in enumerate(images):
    #image path
    img_path=f'{IMAGES_DIR_PATH}/{image}'
    
    #image name
    f_name=image.split('.')[0]

    #getting image size
    im = Image.open(img_path)
    width,height= im.size
    
    #reading image
    img=cv2.imread(img_path)
    
    #detecting license plate using yolov5 model
    [top_x,top_y,bottom_x,bottom_y]=get_yolo_label(f_name)
    
    #cropping the license plate from the original image
    plate=img[top_y+10:bottom_y-10,top_x+5:bottom_x-5]
    
    #apply pre-processing on plate
    plate=pre_process(plate)
    
    #character segmentation
    chars=segment_chars(plate)
    
    #predict label
    label_pred=predict_label(chars,char_recognizer)
    
    #read true label
    f1=open(f'./random-cars-dataset/license_numbers/{f_name}.txt')
    label_true=f1.readline()
    
    #cal score
    score=cal_score(label_pred,label_true)
    
    scores.append(score)
    
    print(f"#{i} {f_name} true: {label_true}, pred: {label_pred}, Similarty: {score[0]} , Edit Distance: {score[1]}")
    
    
#display average scores
final_score(scores)

#0 Cars0 true: KL01CA2555, pred: HKLUJU22555L, Similarty: 0.5454545454545454 , Edit Distance: 6
#1 Cars1 true: PGMN112, pred: PJMN11J, Similarty: 0.7142857142857143 , Edit Distance: 2
#2 Cars102 true: 6861136, pred: 6861136, Similarty: 1.0 , Edit Distance: 0
#3 Cars107 true: MH20BQ20, pred: , Similarty: 0.0 , Edit Distance: 8
#4 Cars108 true: MH01AV8866, pred: HHU1AYHHHH, Similarty: 0.3 , Edit Distance: 7
#5 Cars109 true: CZ17KOD, pred: LZJ7KUU, Similarty: 0.42857142857142855 , Edit Distance: 4
#6 Cars11 true: WOR516K, pred: WUWSJEK, Similarty: 0.2857142857142857 , Edit Distance: 5
#7 Cars111 true: MH20EE7598, pred: MHJUHH75HH, Similarty: 0.4 , Edit Distance: 6
#8 Cars113 true: MPEACHW, pred: MHHAJW, Similarty: 0.46153846153846156 , Edit Distance: 4
#9 Cars116 true: MK3532, pred: HJKQK1, Similarty: 0.16666666666666666 , Edit Distance: 6
#10 Cars118 true: JA62UAR, pred: JAH2UAW, Similarty: 0.7142857142857143 , Edit Distance: 2
#11 Cars12 true: MH12BG7237, pred: HH1ZHG77J7, Similarty: 0.